In [40]:
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import RYGate, CRYGate
from qiskit.visualization import circuit_drawer

from susy_qm import calculate_Hamiltonian

import numpy as np

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime import EstimatorOptions
from qiskit_aer import AerSimulator


from qiskit_ibm_runtime.fake_provider import FakeAlmadenV2

from scipy.stats.qmc import Halton
from scipy.optimize import differential_evolution

In [118]:
options = EstimatorOptions()
options.default_shots = 1024
options.optimization_level = 0

#backend = FakeAlmadenV2()
backend = AerSimulator()
estimator = Estimator(backend, options=options)

In [119]:
estimator.options

max_execution_time,Unset
default_precision,Unset
default_shots,1024
optimization_level,0
resilience_level,Unset
seed_estimator,Unset
experimental,Unset
▸environment,EnvironmentOptions
log_level,'WARNING'
callback,None
job_tags,None


In [120]:
potential = 'AHO'
cutoff = 4

H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]
min_eigenvalue = min(eigenvalues.real)

num_qubits = int(1 + np.log2(cutoff))

observables = SparsePauliOp.from_operator(H)

In [121]:
param_objs = [Parameter(f"θ{i}") for i in range(1)]

qc = QuantumCircuit(num_qubits)   
qc.x(0)    
#qc.ry(param_objs[0], num_qubits-1)
qc.ry(param_objs[0], 2)

qc.draw()

┌───┐   
q_0: ──┤ X ├───
       └───┘   
q_1: ──────────
     ┌────────┐
q_2: ┤ Ry(θ0) ├
     └────────┘

In [60]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pm.run(qc)
mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

In [122]:
def cost_function(params):

    job = estimator.run([(qc, observables, params)])
    energy = job.result()[0].data.evs.sum()

    return energy

In [109]:
iteration_count = 0
def callback(xk, convergence=None):
    global iteration_count
    iteration_count += 1
    if iteration_count % 10 == 0:
        energy= cost_function(xk)
        print(f"Iteration {iteration_count}: Energy = {energy:.8f}")

In [110]:
seed=42
num_params = 1
bounds = [(0, 2 * np.pi) for _ in range(num_params)]

max_iter = 1000
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

halton_sampler = Halton(d=num_params, seed=seed)
scaled_samples = 2 * np.pi * halton_sampler.random(n=popsize)


result = differential_evolution(
    cost_function,
    bounds=bounds,
    maxiter=max_iter,
    tol=tol,
    atol=abs_tol,
    strategy=strategy,
    popsize=popsize,
    init=scaled_samples,
    seed=seed,
    callback=callback
)

Iteration 10: Energy = -0.15903888
Iteration 20: Energy = -0.13362760
Iteration 30: Energy = -0.20001623
Iteration 40: Energy = -0.16398019
Iteration 50: Energy = -0.17295949
Iteration 60: Energy = -0.23500811
Iteration 70: Energy = -0.11092078
Iteration 80: Energy = -0.10405454
Iteration 90: Energy = -0.26679326
Iteration 100: Energy = -0.17508247
Iteration 110: Energy = -0.17985544
Iteration 120: Energy = -0.11943247
Iteration 130: Energy = -0.12101355
Iteration 140: Energy = -0.03985324
Iteration 150: Energy = -0.06694464
Iteration 160: Energy = -0.19471128
Iteration 170: Energy = -0.14963157
Iteration 180: Energy = -0.24348514
Iteration 190: Energy = 0.00672338
Iteration 200: Energy = -0.10776850
Iteration 210: Energy = -0.23818514
Iteration 220: Energy = -0.10673425
Iteration 230: Energy = -0.16982208
Iteration 240: Energy = -0.17823969
Iteration 250: Energy = -0.18781039
Iteration 260: Energy = -0.18569732
Iteration 270: Energy = -0.16556128
Iteration 280: Energy = -0.12317118
It

In [111]:
result.success

False

In [112]:
result.fun

np.float64(-0.3914917991424861)

In [95]:
min_eigenvalue

np.float64(-0.16478526068502247)